In [ ]:
using Pkg
Pkg.add("Images")
Pkg.add("ImageCore")
Pkg.add("JuMP")
Pkg.add("Flux")
Pkg.resolve()

In [ ]:
using MLDatasets
using Images, ImageCore
using LinearAlgebra, Statistics, Plots
using NLopt, JuMP
using Flux

# Classification

We consider the [FashionMNIST](https://github.com/zalandoresearch/fashion-mnist) dataset from [Zalando Research](https://github.com/zalandoresearch).

It contains images of Fashion items from 9 categories.

In Julia this dataset can be loaded through the [MLDatasets](https://github.com/JuliaML/MLDatasets.jl) package.

In [ ]:
X_all, Y_all = FashionMNIST(split=:train)[:];

Now, `X_all` contains the input data and `Y_all` the output data of the training data in the `FashionMNIST` dataset.

The entries of `Y_all` correspond to the following categories:

In [ ]:
labels = ["T-shirt/top", 
            "Trouser", 
            "Pullover", 
            "Dress", 
            "Coat", 
            "Sandal", 
            "Shirt", 
            "Sneaker", 
            "Bag", 
            "Ankle boot"];

Since `Julia` indexing starts at `1` and `Y_all` contains values between `0` and `9`, it will be convenient to add 1 to all values `Y_all`.  

In [ ]:
Y_all = Y_all .+ 1;

Let us take a look at 6 randomly chosen images along with their labels.

In [ ]:
k = rand(1:60000, 6)
println(["$(labels[Y_all[i]])" for i in k])
[convert2image(FashionMNIST, X_all[:,:,i]) for i in k]

## Choosing two categories

To illustrate how the classification methods from the lecture work we consider only two items, namely Coats and Sandals. The goal is to learn an algorithm that can distinguish pictures from these two categories.

First, we compute the locations of Coats and Sandals in the data.

In [ ]:
l₁ = findfirst(labels .== "Coat")
l₂ = findfirst(labels .== "Sandal")
l = findall(y -> y == l₁ || y == l₂, Y_all);
length(l)

I.e., there are 12000 items in the data, which are either Coats or Sandals.

For training let us consider a subsample of size $n=50$. 

Since the goal of this notebook is to illustrate how classification works, we will not stick to the algorithm for approaching machine learning problems. We will simply take the first $n$ items in the data set (instead of choosing them randomly). We also take another $n$ items for testing. 

In [ ]:
n = 50

Now, we split the data into the training and test data

In [ ]:
train_indices = l[1:n];
test_indices = l[n+1:2*n];

X_test = X_all[:, :, test_indices]
Y_test = Y_all[test_indices];
|
X = X_all[:, :, train_indices]
Y = Y_all[train_indices];

Let us look at a subsample of the training data:

In [ ]:
k = rand(1:n, 8)
println(["$(labels[Y[i]])" for i in k])
[convert2image(FashionMNIST, X[:,:,i]) for i in k]

## Visualizing the data

Each input data point is an image consisting of grey values for $28\times 28$ pixels.

Thus the data lives in $\mathbb R^D$ with $D=28^2 = 784$.

In [ ]:
D = 28^2

Let us look at the structure of the first input data point:

In [ ]:
typeof(X[:,:,1])

We see that the input data consists of matrices. 

For using machine learning algorithm we first transform the input data points into vectors using the `vec` function.

In [ ]:
X_vec = [Float64.(vec(X[:,:,i])) for i in 1:n];
X_test_vec = [Float64.(vec(X_test[:,:,i])) for i in 1:n];

We also normalize the data to account for numerical accuracy.

In [ ]:
M = maximum(norm.(X_vec))
X_vec = X_vec ./ M
X_test_vec = X_test_vec ./ M;

We can visualize this data by randomly projecting the points in $\mathbb R^D$ to $\mathbb R^2$. 

For instance, we can first center the data and then sample a random projection matrix:

In [ ]:
x̄ = mean(X_vec)
Pr = randn(2, D)
X_Pr = hcat(map(x -> Pr*(x - x̄), X_vec)...);

We can now apply a scatter plot the the projected data.

In [ ]:
scatter(X_Pr[1, Y .== l₁], X_Pr[2, Y .== l₁], 
            color = :indianred,
            label = labels[l₁],
            size = (800,300))
scatter!(X_Pr[1, Y .== l₂], X_Pr[2, Y .== l₂], 
            color = :steelblue,
            label = labels[l₂],
            size = (600,300))

## Support Vector Machine

The first attempt to learn an algorithm for classification is using an SVM.

We will use the simple kernel that gives back the standard inner product between data points.

In [ ]:
κ(x₁,x₂) = x₁ ⋅ x₂

The kernel defines a kernel matrix as follows:

In [ ]:
G = [κ(x₁,x₂) for (x₁,x₂) in 
        Iterators.product(X_vec,X_vec)];

We also need to transform the output data so that it takes values in $\{-1,1\}$.

In [ ]:
y = map(Y) do Yᵢ
    if Yᵢ==l₁
        -1
    elseif Yᵢ==l₂
        1
    end
end;

In [ ]:
y

We are now ready to set up the optimization problem solving the Dual SVM.

The regularization parameter is set to $C=5$.

We use [JuMP](https://jump.dev/JuMP.jl/stable/) for setting up the model and [NLopt](https://github.com/JuliaOpt/NLopt.jl) for solving the optimization problem.

In [ ]:
C = 5;
model = Model(NLopt.Optimizer)

set_optimizer_attribute(model, "algorithm", :AUGLAG)
set_optimizer_attribute(model, "local_optimizer", :LD_LBFGS)

@variable(model, α[1:n]);

The constraints are defined next:

In [ ]:
@constraint(model, c₁, sum(y .* α) .== 0);
@constraint(model, c₂, α .>= zeros(n));
@constraint(model, c₃, α .<= C .* ones(n));

Finally, we define the objective function.

In [ ]:
@objective(model, Max, 
           4 * sum(α) - sum(y[i]*α[i]*y[j]*α[j]*G[i,j] 
            for (i,j) in Iterators.product(1:n,1:n)));

Using the following command we can now solve the optimization problem 

In [ ]:
JuMP.optimize!(model)

We use the optimal value to define $\psi$ and $b$ for our SVM.

In [ ]:
α_opt = value.(α)

In [ ]:
α_opt = value.(α)
ψ(x) =  sum(0.5 * y[i] * α_opt[i] * κ(x, X_vec[i]) for i in 1:n)
b = median(y[i] - ψ(X_vec[i]) for i in 1:n if abs(α_opt[i]) > 1e-8)

This defines the following function for classification.

In [ ]:
f_class(x) = Int(sign(ψ(x) + b))

We can use `f_class` in a prediction function:

In [ ]:
function predict(f, x)
    if f(x) == -1
        println("This is a $(labels[l₁]):")
    elseif f(x) == 1
         println("This is a $(labels[l₂]):")
    end
end

#### How well does our prediction function work?

We can now check the quality of our prediction function.

For instance, we can randomly choose items from the test data and let our algorithm predict what item it is.

In [ ]:
ℓ = rand(1:n)
predict(f_class, X_test_vec[ℓ])
convert2image(FashionMNIST, X_test[:,:,ℓ])

As a second check, we plot projected test data points along with our predictions. 


In [ ]:
x̄_test = mean(X_test_vec);
X_test_Pr = hcat(map(x -> Pr*(x - x̄_test), X_test_vec)...);


For this, we produce $y_{\mathrm{test}}\in\{-1,1\}$ for the true labels and $\hat{y}_{\mathrm{test}}$ is the vector of predictions 

In [ ]:
y_test = map(Y_test) do y
    if y==l₁
        -1
    elseif y==l₂
        1
    end
    end;
ŷ_test = map(f_class, X_test_vec);

The visualization comes next:

In [ ]:
scatter(X_test_Pr[1, y_test .== -1], X_test_Pr[2, y_test .== -1], 
            color = :indianred,
            label = labels[l₁],
            size = (600,300),
            subplot = 1, layout = 2,
            xlabel = "true labels test data")
scatter!(X_test_Pr[1, y_test .== 1], X_test_Pr[2, y_test .== 1], 
            color = :steelblue,
            label = labels[l₂],
            subplot = 1, layout = 2)
scatter!(X_test_Pr[1, ŷ_test .== -1], X_test_Pr[2, ŷ_test .== -1], 
            color = :indianred,
            label = labels[l₁],
            size = (600,300),
            subplot = 2, layout = 2,
            xlabel = "predictions test data")
scatter!(X_test_Pr[1, ŷ_test .== 1], X_test_Pr[2, ŷ_test .== 1], 
            color = :steelblue,
            label = labels[l₂],
            subplot = 2, layout = 2)

The fraction of uncorrectly prediced data points is

In [ ]:
count((y_test-ŷ_test) .!= 0)/n

# Neural Networks

We will now compute a classification algorithm using a neural network.

Instead of setting up the network by hand as we did in Notebook 4, we will use [Flux.jl](https://fluxml.ai).

We set up a neural network with three layers.

The first layer is $\mathbb R^D\to\mathbb R^{N_1}$. 

The second layer is $\mathbb R^{N_1}\to\mathbb R^{N_2}$. 

The third layer is $\mathbb R^{N_2}\to\mathbb R^{2}$.

The output in $\mathbb R^{2}$ will be a discrete probability distribution for our two items. Therefore, we use as the activation function in the last layer the softmax function. The other activation functions are chosen to be the ReLu function.

In [ ]:
N₁, N₂ = 20, 15

NN = Flux.Chain(
          Dense(28^2, N₁, relu),
          Dense(N₁, N₂, relu),
          Dense(N₂, 2),
          softmax)

We have to modify the output data accordingly.

In [ ]:
y₀ = map(Y) do Yᵢ
    if Yᵢ==l₁
        [1;0]
    elseif Yᵢ==l₂
        [0;1]
    end
    end;

As loss function we take the [crossentropy](https://en.wikipedia.org/wiki/Cross_entropy) function `ℓ(p,q) = -p₁ log(q₁) - p₂ log(q₂).`

In [ ]:
loss(x, y) = Flux.crossentropy(NN(x), y; dims = 2);
# in Flux.jl the first variable is in ℝ^D

The framework to set up a Flux.jl' model is as follows:

(The code is adapted from the lecture on [Neural Nets](https://github.com/JuliaAcademy/DataScience/blob/main/10.%20Neural%20Nets.ipynb) by [Huda Nassar](https://github.com/nassarhuda).)

In [ ]:
ps = Flux.params(NN);
dataset = [(Float32.(x),y) for (x,y) in zip(X_vec, y₀)];
opt = ADAM();

The model is the trained using Flux.jl's `train!` function.

In [ ]:
#Flux.@epochs 3 Flux.train!(loss, ps, dataset, opt)
for i in 1:3
    Flux.train!(loss, ps, dataset, opt) #train for 3 data loops, one loop is called an epoch, @epoch is outdated
end

Let us now see what the algorithm that we have learned predicts on randomly chosen elements from the test data set.

In [ ]:
ℓ = rand(1:n)
p = NN(X_test_vec[ℓ])
println("With probability $(p[1]): Coat.\n")
println("With probability $(p[2]): Sandal.\n")
convert2image(FashionMNIST, X_test[:,:,ℓ])

Finally, we consider again the projected data and compare the labels from the neural network to the true labels on the test data.

The size of the points should be proportional to the probability returned from the neural network.

In [ ]:
sizes = map(x->maximum(NN(x)), X_test_vec) .* 6
ŷ₀_test = map(x->argmax(NN(x)), X_test_vec);

In [ ]:
scatter(X_test_Pr[1, y_test .== -1], X_test_Pr[2, y_test .== -1], 
            color = :indianred,
            label = labels[l₁],
            size = (600,300),
            subplot = 1, layout = 2,
            xlabel = "true labels test data")
scatter!(X_test_Pr[1, y_test .== 1], X_test_Pr[2, y_test .== 1], 
            color = :steelblue,
            label = labels[l₂],
            subplot = 1, layout = 2)
scatter!(X_test_Pr[1, ŷ₀_test .== 1], X_test_Pr[2, ŷ₀_test .== 1], 
            color = :indianred,
            label = labels[l₁], subplot = 2,
            size = (600,300), markersizes = sizes[ŷ₀_test .== 1],
            xlabel = "predictions test data")
scatter!(X_test_Pr[1, ŷ₀_test .== 2], X_test_Pr[2, ŷ₀_test .== 2], 
            color = :steelblue, subplot = 2, markersizes = sizes[ŷ₀_test .== 2],
            label = labels[l₂])

In [ ]:
count((y_test- 2 .* (ŷ₀_test .- 1.5)) .!= 0)/n